# Timesheet Overlap Calculation (Odoo)

**Use Case:** Before creating a new timesheet in Odoo, check if it overlaps with any existing timesheets for the same user.

**How it works:**
1. Connect to Odoo via XML-RPC
2. Search existing `account.analytic.line` records for the user
3. Vectorized overlap check: `A_start < B_end AND B_start < A_end`

## 1. Odoo Connection Config

In [ ]:
import xmlrpc.client
import pandas as pd
from datetime import datetime, timedelta

# ═══ Odoo Connection Settings ═══
ODOO_URL = 'http://localhost:8069' # Change to your Odoo URL
ODOO_DB = 'database_name'          # Change to your database name
ODOO_USER = 'admin'                # Change to your username
ODOO_PASSWORD = 'admin'            # Change to your password

# Connect
common = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/common')
uid = common.authenticate(ODOO_DB, ODOO_USER, ODOO_PASSWORD, {})
models = xmlrpc.client.ServerProxy(f'{ODOO_URL}/xmlrpc/2/object')

if uid:
    print(f'✅ Connected to Odoo as uid={uid}')
else:
    print('❌ Authentication failed — check credentials')

✅ Connected to Odoo as uid=10


## 2. Core Functions

In [65]:
def search_timesheets(user_id: int, date_from: str = None, date_to: str = None) -> pd.DataFrame:
    """
    Search existing timesheets from Odoo for a specific user.
    
    Efficiently narrows the search by date range to avoid loading all records.
    
    Parameters:
        user_id:   Odoo user ID (res.users)
        date_from: Optional start date filter 'YYYY-MM-DD' (defaults to 30 days ago)
        date_to:   Optional end date filter 'YYYY-MM-DD' (defaults to 30 days from now)
    
    Returns:
        DataFrame with columns: [id, name, date_time, date_time_end, user_id, employee_id]
    """
    if not date_from:
        date_from = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d 00:00:00')
    if not date_to:
        date_to = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d 23:59:59')
    
    domain = [
        ['user_id', '=', user_id],
        ['date_time', '>=', date_from],
        ['date_time', '<=', date_to],
    ]
    
    fields = ['id', 'name', 'date', 'unit_amount', 'user_id', 'employee_id']
    
    records = models.execute_kw(
        ODOO_DB, uid, ODOO_PASSWORD,
        'account.analytic.line', 'search_read',
        [domain],
        {'fields': fields, 'order': 'date asc'}
    )
    
    if not records:
        print(f'ℹ️ No timesheets found for user_id={user_id} between {date_from[:10]} and {date_to[:10]}')
        return pd.DataFrame(columns=['id', 'name', 'date', 'unit_amount', 'user_id', 'employee_id'])
    
    df = pd.DataFrame(records)
    print(f'📊 Found {len(df)} timesheets for user_id={user_id}')
    return df

In [56]:
def search_timesheets_with_time(user_id: int, date_from: str = None, date_to: str = None) -> pd.DataFrame:
    """
    Search existing timesheets that have date_time and date_time_end fields.
    (Available in Odoo 17+ or with timesheet grid / start-end time modules)
    
    Parameters:
        user_id:   Odoo user ID (res.users)
        date_from: Optional start date filter 'YYYY-MM-DD'
        date_to:   Optional end date filter 'YYYY-MM-DD'
    
    Returns:
        DataFrame with columns: [id, name, date_time, date_time_end, user_id]
    """
    if not date_from:
        date_from = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d 00:00:00')
    if not date_to:
        date_to = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d 23:59:59')
    
    # Try to use date_time fields (Odoo 17+ or timesheet modules with start/end)
    # Adjust field names based on your Odoo version/modules:
    #   - Odoo Enterprise: 'date_time', 'date_time_end' (from timesheet_grid)
    #   - Custom modules: may use 'time_start', 'time_stop', etc.
    domain = [
        ['user_id', '=', user_id],
        ['date_time', '>=', date_from],
        ['date_time', '<=', date_to],
    ]
    
    fields = ['id', 'name', 'date_time', 'date_time_end', 'user_id', 'employee_id']
    
    records = models.execute_kw(
        ODOO_DB, uid, ODOO_PASSWORD,
        'account.analytic.line', 'search_read',
        [domain],
        {'fields': fields, 'order': 'date_time asc'}
    )
    
    if not records:
        print(f'ℹ️ No timesheets found for user_id={user_id}')
        return pd.DataFrame(columns=['id', 'name', 'date_time', 'date_time_end', 'user_id'])
    
    df = pd.DataFrame(records)
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['date_time_end'] = pd.to_datetime(df['date_time_end'])
    
    print(f'📊 Found {len(df)} timesheets for user_id={user_id}')
    return df

In [57]:
def check_overlap(
    user_id: int,
    new_start: str,
    new_end: str,
    new_name: str = 'New Entry'
):
    """
    Check if a new timesheet entry overlaps with existing entries in Odoo.
    
    1. Searches Odoo for existing timesheets (filtered by user + date range)
    2. Vectorized overlap check against all results
    
    Parameters:
        user_id:   Odoo user ID
        new_start: Start datetime 'YYYY-MM-DD HH:MM:SS'
        new_end:   End datetime 'YYYY-MM-DD HH:MM:SS'
        new_name:  Name/description of the new entry
    
    Returns:
        (is_valid: bool, conflicts: DataFrame or None, existing: DataFrame)
    """
    new_start_dt = pd.to_datetime(new_start)
    new_end_dt = pd.to_datetime(new_end)
    
    if new_start_dt >= new_end_dt:
        raise ValueError(f'Start ({new_start}) must be before end ({new_end})')
    
    print(f"📝 Checking: '{new_name}'")
    print(f"   User ID: {user_id}")
    print(f"   Time:    {new_start} → {new_end}")
    print()
    
    # Search Odoo — only fetch timesheets around the target date
    date_from = (new_start_dt - timedelta(days=1)).strftime('%Y-%m-%d 00:00:00')
    date_to = (new_end_dt + timedelta(days=1)).strftime('%Y-%m-%d 23:59:59')
    
    existing = search_timesheets_with_time(user_id, date_from, date_to)
    
    if existing.empty:
        print('   ✅ No existing timesheets — no overlap possible!')
        return True, None, existing
    
    # Vectorized overlap: A_start < B_end AND B_start < A_end
    conflicts = existing[
        (existing['date_time'] < new_end_dt) &
        (new_start_dt < existing['date_time_end'])
    ].copy()
    
    if conflicts.empty:
        print(f'   ✅ No overlap with {len(existing)} existing timesheets!')
        return True, None, existing
    
    # Calculate overlap details
    conflicts['overlap_start'] = conflicts['date_time'].clip(lower=new_start_dt)
    conflicts['overlap_end'] = conflicts['date_time_end'].clip(upper=new_end_dt)
    conflicts['overlap_minutes'] = (
        (conflicts['overlap_end'] - conflicts['overlap_start']).dt.total_seconds() / 60
    ).astype(int)
    
    print(f'   ❌ OVERLAP DETECTED with {len(conflicts)} timesheet(s)!')
    print()
    for _, c in conflicts.iterrows():
        print(f"   ⚠️  id={c['id']} '{c['name']}'")
        print(f"       Time: {c['date_time'].strftime('%Y-%m-%d %H:%M')} → {c['date_time_end'].strftime('%Y-%m-%d %H:%M')}")
        print(f"       Overlap: {c['overlap_minutes']} min ({c['overlap_start'].strftime('%H:%M')} - {c['overlap_end'].strftime('%H:%M')})")
    
    return False, conflicts, existing

## 3. View Existing Timesheets for a User

In [68]:
# ═══ Change this to the user you want to check ═══
TARGET_USER_ID = 10   # Odoo user ID (res.users)
TARGET_DATE = '2026-02-26'

existing = search_timesheets_with_time(
    user_id=TARGET_USER_ID,
    date_from=f'{TARGET_DATE} 00:00:00',
    date_to=f'{TARGET_DATE} 23:59:59'
)
existing

📊 Found 1 timesheets for user_id=10


,id,name,date_time,date_time_end,user_id,employee_id
0,151377,TEST,2026-02-26 02:41:25,2026-02-26 03:41:33,"[10, Rifqi Oktabhiar Erawan]","[494, [981220] Rifqi Oktabhiar Erawan]"


## 4. Test: Create Timesheet — NO Overlap (Should Pass)

In [69]:
# ═══ This should PASS — adjust times to fit a gap in the user's schedule ═══
is_valid, conflicts, _ = check_overlap(
    user_id=TARGET_USER_ID,
    new_start='2026-02-26 17:30:00',
    new_end='2026-02-26 19:00:00',
    new_name='Overtime: deploy fix'
)

print()
print(f'Result: {"✅ CAN CREATE" if is_valid else "❌ CANNOT CREATE"}')

📝 Checking: 'Overtime: deploy fix'
   User ID: 10
   Time:    2026-02-26 17:30:00 → 2026-02-26 19:00:00

📊 Found 1 timesheets for user_id=10
   ✅ No overlap with 1 existing timesheets!

Result: ✅ CAN CREATE


## 5. Test: Create Timesheet — OVERLAP (Should Fail)

In [71]:
# ═══ This should FAIL — overlaps with an existing entry ═══
is_valid, conflicts, _ = check_overlap(
    user_id=TARGET_USER_ID,
    new_start='2026-02-26 01:00:00',
    new_end='2026-02-26 03:30:00',
    new_name='Urgent call with PM'
)

print()
print(f'Result: {"✅ CAN CREATE" if is_valid else "❌ CANNOT CREATE"}')
if conflicts is not None:
    print(f'Conflicting entries:')
    display(conflicts[['id', 'name', 'date_time', 'date_time_end', 'overlap_minutes']])

📝 Checking: 'Urgent call with PM'
   User ID: 10
   Time:    2026-02-26 01:00:00 → 2026-02-26 03:30:00

📊 Found 1 timesheets for user_id=10
   ❌ OVERLAP DETECTED with 1 timesheet(s)!

   ⚠️  id=151377 'TEST'
       Time: 2026-02-26 02:41 → 2026-02-26 03:41
       Overlap: 48 min (02:41 - 03:30)

Result: ❌ CANNOT CREATE
Conflicting entries:


,id,name,date_time,date_time_end,overlap_minutes
0,151377,TEST,2026-02-26 02:41:25,2026-02-26 03:41:33,48


## 6. Test: Overlap Spanning Multiple Entries

In [61]:
# ═══ This should FAIL — spans across multiple existing entries ═══
is_valid, conflicts, _ = check_overlap(
    user_id=TARGET_USER_ID,
    new_start='2026-02-26 10:30:00',
    new_end='2026-02-26 13:30:00',
    new_name='Long workshop'
)

print()
print(f'Result: {"✅ CAN CREATE" if is_valid else "❌ CANNOT CREATE"}')
if conflicts is not None:
    display(conflicts[['id', 'name', 'date_time', 'date_time_end', 'overlap_minutes']])

📝 Checking: 'Long workshop'
   User ID: 2
   Time:    2026-02-26 10:30:00 → 2026-02-26 13:30:00

ℹ️ No timesheets found for user_id=2
   ✅ No existing timesheets — no overlap possible!

Result: ✅ CAN CREATE


## 7. Batch Check — Multiple New Entries at Once

In [62]:
# Check multiple proposed entries in one go
proposed_entries = [
    {'name': 'Morning coffee chat',  'start': '2026-02-26 07:30:00', 'end': '2026-02-26 08:00:00'},
    {'name': 'Overlap test entry',   'start': '2026-02-26 09:00:00', 'end': '2026-02-26 10:00:00'},
    {'name': 'Evening training',     'start': '2026-02-26 18:00:00', 'end': '2026-02-26 19:30:00'},
    {'name': 'Late night work',      'start': '2026-02-26 22:00:00', 'end': '2026-02-26 23:00:00'},
]

print(f'Batch checking {len(proposed_entries)} entries for user_id={TARGET_USER_ID}')
print('=' * 65)

results = []
for entry in proposed_entries:
    print()
    is_valid, conflicts, _ = check_overlap(
        user_id=TARGET_USER_ID,
        new_start=entry['start'],
        new_end=entry['end'],
        new_name=entry['name']
    )
    results.append({
        'name': entry['name'],
        'start': entry['start'],
        'end': entry['end'],
        'can_create': is_valid,
        'conflicts_count': len(conflicts) if conflicts is not None else 0
    })
    print('─' * 65)

print()
print('📊 Batch Results:')
results_df = pd.DataFrame(results)
results_df['status'] = results_df['can_create'].map({True: '✅ OK', False: '❌ Overlap'})
results_df

Batch checking 4 entries for user_id=2

📝 Checking: 'Morning coffee chat'
   User ID: 2
   Time:    2026-02-26 07:30:00 → 2026-02-26 08:00:00

ℹ️ No timesheets found for user_id=2
   ✅ No existing timesheets — no overlap possible!
─────────────────────────────────────────────────────────────────

📝 Checking: 'Overlap test entry'
   User ID: 2
   Time:    2026-02-26 09:00:00 → 2026-02-26 10:00:00

ℹ️ No timesheets found for user_id=2
   ✅ No existing timesheets — no overlap possible!
─────────────────────────────────────────────────────────────────

📝 Checking: 'Evening training'
   User ID: 2
   Time:    2026-02-26 18:00:00 → 2026-02-26 19:30:00

ℹ️ No timesheets found for user_id=2
   ✅ No existing timesheets — no overlap possible!
─────────────────────────────────────────────────────────────────

📝 Checking: 'Late night work'
   User ID: 2
   Time:    2026-02-26 22:00:00 → 2026-02-26 23:00:00

ℹ️ No timesheets found for user_id=2
   ✅ No existing timesheets — no overlap possible!
───

,name,start,end,can_create,conflicts_count,status
0,Morning coffee chat,2026-02-26 07:30:00,2026-02-26 08:00:00,True,0,✅ OK
1,Overlap test entry,2026-02-26 09:00:00,2026-02-26 10:00:00,True,0,✅ OK
2,Evening training,2026-02-26 18:00:00,2026-02-26 19:30:00,True,0,✅ OK
3,Late night work,2026-02-26 22:00:00,2026-02-26 23:00:00,True,0,✅ OK


## Notes

### Odoo Field Names
The `date_time` and `date_time_end` fields are available in:
- **Odoo 17+ Enterprise** (via `timesheet_grid` module)
- **Custom modules** that add start/end time to timesheets

If your Odoo version uses different field names, update the `search_timesheets_with_time()` function.

Common alternatives:
```python
# Some modules use:
fields = ['time_start', 'time_stop']  
# Or computed from date + unit_amount:
fields = ['date', 'unit_amount']  # end = date + unit_amount hours
```

### Efficiency
- **Server-side filtering**: The Odoo search domain filters by `user_id` + date range, so only relevant records are fetched
- **Narrow date window**: `check_overlap()` only queries ±1 day around the new entry's date
- **Vectorized check**: Pandas boolean indexing, no Python loops